In [1]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

In [2]:
import numpy as np
import os

In [3]:
import tensorflow as tf
from tensorflow import keras

2022-02-08 10:03:43.773187: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/benson516/agv_ws/devel/lib:/opt/ros/noetic/lib
2022-02-08 10:03:43.773220: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Saving a TensorFlow Model
## Using Tensorflow Serving
### Exporting SaveModels

Prepare data

In [4]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

11501568/11490434 [==============================] - 5s 0us/step


Prepare and train a model

In [5]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

2022-02-08 10:03:56.862203: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-08 10:03:56.862402: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (benson516-P1G2): /proc/driver/nvidia/version does not exist
2022-02-08 10:03:56.866207: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/benson516/ml/my_env/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2022-02-08 10:03:57.627334: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the M

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7012 - accuracy: 0.8241 - val_loss: 0.3715 - val_accuracy: 0.9024
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3536 - accuracy: 0.9020 - val_loss: 0.2990 - val_accuracy: 0.9144
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3036 - accuracy: 0.9145 - val_loss: 0.2651 - val_accuracy: 0.9272
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2736 - accuracy: 0.9231 - val_loss: 0.2436 - val_accuracy: 0.9334
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2509 - accuracy: 0.9296 - val_loss: 0.2257 - val_accuracy: 0.9364
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2322 - accuracy: 0.9350 - val_loss: 0.2121 - val_accuracy: 0.9396
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2161 - accuracy: 0.9400 - val_loss: 0.1970 - val_accuracy:

In [6]:
np.round(model.predict(X_new), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [7]:
# Save the model to Tensorflow SaveModel format
model_version = "0001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)

# --- Legacy method (not be able to be load by keras)
# tf.saved_model.save(model, model_path)

# --- New method (can be load by keras)
keras.models.save_model(model, model_path)
# model.save(model_path)

2022-02-08 10:04:50.967801: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


In [8]:
# Load the model: 1. using tf.saved_model.load --> function/object
saved_model = tf.saved_model.load(model_path)

# Make prediction by simply make a function call
y_pred = saved_model(tf.constant(X_new, dtype=tf.float32))
print(y_pred)

tf.Tensor(
[[1.1465363e-04 1.5234058e-07 9.7585790e-04 2.7667386e-03 3.7489147e-06
  7.6716256e-05 3.9241218e-08 9.9557823e-01 5.3566997e-05 4.3032545e-04]
 [8.1256160e-04 3.5265093e-05 9.8830998e-01 7.0190392e-03 1.2800497e-07
  2.3161969e-04 2.5624933e-03 9.7052555e-10 1.0287794e-03 8.7537394e-08]
 [4.4473782e-05 9.7030050e-01 9.0676993e-03 2.2686215e-03 4.8565315e-04
  2.8726975e-03 2.2721561e-03 8.3507244e-03 4.0399102e-03 2.9759880e-04]], shape=(3, 10), dtype=float32)


In [9]:
# Load the model: 2. using keras.models.load_model --> keras model
model = keras.models.load_model(model_path)

# Make prediction as usual (it's a normal keras model object) 
y_pred = model.predict(tf.constant(X_new, dtype=tf.float32))
print(y_pred)

[[1.1465363e-04 1.5234058e-07 9.7585790e-04 2.7667386e-03 3.7489147e-06
  7.6716256e-05 3.9241218e-08 9.9557823e-01 5.3566997e-05 4.3032545e-04]
 [8.1256160e-04 3.5265093e-05 9.8830998e-01 7.0190392e-03 1.2800497e-07
  2.3161969e-04 2.5624933e-03 9.7052555e-10 1.0287794e-03 8.7537394e-08]
 [4.4473782e-05 9.7030050e-01 9.0676993e-03 2.2686215e-03 4.8565315e-04
  2.8726975e-03 2.2721561e-03 8.3507244e-03 4.0399102e-03 2.9759880e-04]]


In [10]:
!saved_model_cli show --dir {model_path} --all

2022-02-08 10:04:52.692801: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/benson516/agv_ws/devel/lib:/opt/ros/noetic/lib
2022-02-08 10:04:52.692839: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:


Test in command line

In [11]:
np.save("my_mnist_tests.npy", X_new)

In [12]:
!saved_model_cli run --dir {model_path} --tag_set serve --signature_def serving_default --inputs flatten_input=my_mnist_tests.npy

2022-02-08 10:04:56.322962: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/benson516/agv_ws/devel/lib:/opt/ros/noetic/lib
2022-02-08 10:04:56.323122: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-02-08 10:05:01.828218: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-08 10:05:01.828343: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (benson516-P1G2): /proc/driver/nvidia/version does not exist
2022-02-08 10:05:01.829858: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use t

### Querying TF Serving through the REST API

In [14]:
import json

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [15]:
input_data_json

'{"signature_name": "serving_default", "instances": [[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0],

In [19]:
import requests

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

print(response)

{'predictions': [[0.000114653631, 1.52340576e-07, 0.000975857896, 0.00276673865, 3.74891465e-06, 7.67162564e-05, 3.92412183e-08, 0.995578229, 5.35669969e-05, 0.000430325454], [0.0008125616, 3.52650932e-05, 0.98831, 0.00701903924, 1.2800497e-07, 0.000231619691, 0.00256249332, 9.70525549e-10, 0.00102877943, 8.75373942e-08], [4.44737816e-05, 0.970300496, 0.00906769931, 0.00226862147, 0.000485653145, 0.00287269754, 0.00227215607, 0.00835072435, 0.00403991, 0.000297598803]]}


In [20]:
y_proba = np.array(response["predictions"])
print(y_proba.round(2))

[[0.   0.   0.   0.   0.   0.   0.   1.   0.   0.  ]
 [0.   0.   0.99 0.01 0.   0.   0.   0.   0.   0.  ]
 [0.   0.97 0.01 0.   0.   0.   0.   0.01 0.   0.  ]]


### Querying TF Serving through the gRPC API

In [26]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [27]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

In [33]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto) # Note: this is already a numpy ndarray
print(y_proba)

[[1.1465363e-04 1.5234058e-07 9.7585790e-04 2.7667386e-03 3.7489147e-06
  7.6716256e-05 3.9241218e-08 9.9557823e-01 5.3566997e-05 4.3032545e-04]
 [8.1256160e-04 3.5265093e-05 9.8830998e-01 7.0190392e-03 1.2800497e-07
  2.3161969e-04 2.5624933e-03 9.7052555e-10 1.0287794e-03 8.7537394e-08]
 [4.4473782e-05 9.7030050e-01 9.0676993e-03 2.2686215e-03 4.8565315e-04
  2.8726975e-03 2.2721561e-03 8.3507244e-03 4.0399102e-03 2.9759880e-04]]


In [34]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

### Deploying a new model version

A new model (ver.0002)

In [35]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

/home/benson516/ml/my_env/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7012 - accuracy: 0.8241 - val_loss: 0.3715 - val_accuracy: 0.9024
Epoch 2/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3536 - accuracy: 0.9020 - val_loss: 0.2990 - val_accuracy: 0.9144
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3036 - accuracy: 0.9145 - val_loss: 0.2651 - val_accuracy: 0.9272
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2736 - accuracy: 0.9231 - val_loss: 0.2436 - val_accuracy: 0.9334
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2509 - accuracy: 0.9296 - val_loss: 0.2257 - val_accuracy: 0.9364
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2322 - accuracy: 0.9350 - val_loss: 0.2121 - val_accuracy: 0.9396
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2161 - accuracy: 0.9400 - val_loss: 0.1970 - val_accuracy:

In [36]:
model_version = "0002"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets
